# ДЗ 2. Бейзлайны и детерминированные алгоритмы item-item

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
# Разобьем данные на трейн и тест, где для теста возьмем последние 3 недели
test_size_weeks = 3
data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]
data_train.shape[0], data_test.shape[0]

(2278490, 118314)

In [4]:
# Посчитаем общую скидку
data_train['all_disc'] = np.abs(round(data_train['coupon_disc'] + data_train['coupon_match_disc'] + data_train['retail_disc']))
# Посчитаем выручку
data_train['revenue'] = round(data_train['quantity'] * data_train['sales_value'])

data_train.all_disc.sort_values().nunique(), data_train.revenue.sort_values().nunique() 

<ipython-input-4-850efec8e82e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train['all_disc'] = np.abs(round(data_train['coupon_disc'] + data_train['coupon_match_disc'] + data_train['retail_disc']))
<ipython-input-4-850efec8e82e>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train['revenue'] = round(data_train['quantity'] * data_train['sales_value'])


(71, 17094)

In [5]:
# Группировка купленных товаров по пользователям
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
# Переименование колонок
result.columns=['user_id', 'actual']
# Преобразование в списки
result['actual'] = result['actual'].apply(lambda x: list(x))
result.head(10)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412..."
5,9,"[864335, 990865, 1029743, 9297474, 10457112, 8..."
6,13,"[6534178, 1104146, 829197, 840361, 862070, 884..."
7,14,"[840601, 867293, 933067, 951590, 952408, 96569..."
8,15,"[910439, 1082185, 959076, 1023958, 1082310, 13..."
9,16,"[1062973, 1082185, 13007710]"


# Оценивание
За выполнени каждого задания 1 балл

4 балла -> отл

3 балла -> хор

И тд

### Задание 0. Товар 999999
**На вебинаре мы использовали товар 999999 - что это за товар?**  
Это товар, который не вошел в наш ТОП-5000.

**Зачем он нужен?**  
Чтобы не потерять пользователей, которые купили товары не вошедшие в выборку.

**Используя этот товар мы смещаем качество рекомендаций. В какую сторону?**  
Качество рекомендаций смещается в сторону товара 999999, что ухудшает Precision, т.к. товар будет присутствовать в большинстве рекомендаций.

**Можно ли удалить этот товар?**   
Да, можно.  

Уберите этот товар и сравните с качеством на семинаре.

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. вес = log(sales_sum товара)
- Придумайте пример 3 весов, посчитайте weighted_random_recommendation для разных весов

In [7]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    items = np.array(items_weights['item_id'])
    recs = np.random.choice(items, size=n, replace=False, p=items_weights['weight'].tolist())
    
    return recs.tolist()

Сделайте предсказания

In [8]:
%%time
# Рекомендации для увеличения доли прибыли
items_weights = data_train.groupby('item_id')['revenue'].sum().reset_index()
items_weights = items_weights[items_weights['revenue']>= 10000]
items_weights['weight'] = items_weights['revenue'] / items_weights['revenue'].sum()
items_weights.drop('revenue', axis=1, inplace=True)

Wall time: 79.8 ms


In [9]:
result['weighted_random_recom_total'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))
result.head(2)

,user_id,actual,weighted_random_recom_total
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[397896, 6534178, 6534166, 1426702, 6533889]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6534166, 6534178, 6533889, 707683, 6544236]"


In [10]:
%%time
# Рекомендации по количеству проданных штук
items_weights = data_train.groupby('item_id')['quantity'].sum().reset_index()
items_weights = items_weights[items_weights['quantity']>= 1000]
items_weights['weight'] = items_weights['quantity'] / items_weights['quantity'].sum()
items_weights.drop('quantity', axis=1, inplace=True)

Wall time: 90.8 ms


In [11]:
result['weighted_random_recom_quantity'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))
result.head(2)

,user_id,actual,weighted_random_recom_total,weighted_random_recom_quantity
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[397896, 6534178, 6534166, 1426702, 6533889]","[6534178, 6533889, 1404121, 6534166, 480014]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6534166, 6534178, 6533889, 707683, 6544236]","[6534178, 6534166, 908531, 6533889, 1426702]"


In [12]:
%%time
# Рекомендации по размеру скидки
items_weights = data_train.groupby('item_id')['all_disc'].sum().reset_index()
items_weights = items_weights[items_weights['all_disc']>= 50]
items_weights['weight'] = items_weights['all_disc'] / items_weights['all_disc'].sum()
items_weights.drop('all_disc', axis=1, inplace=True)

Wall time: 80.9 ms


In [13]:
result['weighted_random_recom_disc'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))
result.head(2)

,user_id,actual,weighted_random_recom_total,weighted_random_recom_quantity,weighted_random_recom_disc
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[397896, 6534178, 6534166, 1426702, 6533889]","[6534178, 6533889, 1404121, 6534166, 480014]","[1062782, 13213082, 5978656, 1058214, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6534166, 6534178, 6533889, 707683, 6544236]","[6534178, 6534166, 908531, 6533889, 1426702]","[854405, 995628, 6391557, 995242, 5569230]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма (с вебинара и weighted_random_recommendation) с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество? Почему?

In [14]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]

    flags = np.isin(bought_list, recommended_list)
    precision = flags.sum() / len(recommended_list)

    return precision

In [15]:
result.head(2)

,user_id,actual,weighted_random_recom_total,weighted_random_recom_quantity,weighted_random_recom_disc
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[397896, 6534178, 6534166, 1426702, 6533889]","[6534178, 6533889, 1404121, 6534166, 480014]","[1062782, 13213082, 5978656, 1058214, 1082185]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6534166, 6534178, 6533889, 707683, 6544236]","[6534178, 6534166, 908531, 6533889, 1426702]","[854405, 995628, 6391557, 995242, 5569230]"


In [16]:
precision_at = pd.DataFrame(index=['precision@5'])
for i in result.columns[2:]:
    res = result.apply(lambda x: precision_at_k(x[i], x['actual'],  5), axis=1)
    precision_at[i] = round(res.mean(), 3)

precision_at

,weighted_random_recom_total,weighted_random_recom_quantity,weighted_random_recom_disc
precision@5,0.046,0.049,0.024


In [19]:
# Загрузка predict с семианара
result = pd.read_csv('predictions_basic.csv')
result.head(5)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5588612, 7410370, 1088191, 935577, 6960654]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6552406, 974311, 154857, 6632301, 1417737]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[1096842, 881283, 1135337, 1775272, 9677620]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 878996]","[999999, 1082185, 1029743, 6534178, 1127831]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1084412, 15716900, 405220, 16769955, 888575]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 981760, 999999, 1127831, 961554]","[1082185, 981760, 1127831, 999999, 961554]","[999999, 1082185, 1029743, 1127831, 995785]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[6772768, 858393, 1340682, 6463403, 1090081]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 1098066]","[1082185, 981760, 999999, 1098066, 826249]","[1082185, 981760, 999999, 1098066, 826249]","[999999, 1082185, 1029743, 1098066, 6534178]"


In [18]:
result['actual'] = result['actual'].map(lambda x: x[1:-1].split(', ')).apply(lambda x: list(map(int, x)))

for i in result.columns[2:]:
    result[i] = result[i].map(lambda x: x[1:-1].split(', ')).apply(lambda x: list(map(int, x)))
    res = result.apply(lambda x: precision_at_k(x[i], x['actual'],  5), axis=1)
    precision_at[i] = round(res.mean(), 3)

precision_at.T

,precision@5
weighted_random_recom_total,0.046
weighted_random_recom_quantity,0.049
weighted_random_recom_disc,0.024
random_recommendation,0.000
popular_recommendation,0.155
itemitem,0.137
cosine,0.133
tfidf,0.139
own_purchases,0.180


### Задание 3. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.
- Попробуйте стратегии ансамблирования изученных алгоритмов


In [20]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [24]:
%%time

items = data_train.groupby('item_id')['quantity'].sum().reset_index().\
            sort_values('quantity', ascending=False).head(5000).item_id.tolist()

result['random_recommendation_5000'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))
result.head(2)

Wall time: 672 ms


,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases,random_recommendation_5000
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[5588612, 7410370, 1088191, 935577, 6960654]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1127831, 995242]","[1082185, 999999, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 999999, 1098066]","[999999, 1082185, 1029743, 995785, 1004906]","[970760, 7410168, 1135868, 934131, 6391532]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[6552406, 974311, 154857, 6632301, 1417737]","[6534178, 6533889, 1029743, 6534166, 1082185]","[999999, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 999999, 826249]","[1082185, 981760, 1098066, 826249, 999999]","[999999, 1082185, 1098066, 6534178, 1127831]","[1098248, 873622, 921920, 1029915, 1092945]"


### Задание 4*. Улучшение детерминированных алгоритмов
На семинаре мы рассматривали 



Далее $U \equiv N_i(u) $

$$r_{u,i} =  \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)r_{v, i}$$
$$ S = \sum\limits_{v \in U} \operatorname{sim}(u,v)$$

Предлагается улучшить эту формулу и учесть средние предпочтения всех пользователей

$$r_{u,i} = \mu + \bar{r_u} + \frac{1}{S}\sum\limits_{v \in U}\operatorname{sim}(u,v)(r_{v, i}-\bar{r_{v}} - \mu)$$

Какие смысл имееют $ \mu $ и $ \bar{r_u}$ ?

Реализуйте алгоритм, прогнозирующий рейтинги на основе данной формулы, на numpy (векторизованно!)

В качестве схожести возьмите CosineSimilarity.

Примените к user_item_matrix. В качестве рейтингов возьмите количество или стоимость купленного товара. 
Данный алгоритм предсказывает рейтинги. Как на основании предсказанных рейтингов предсказать факт покупки?

Предложите вариант.
Посчитайте accuracy@5 и сравните с алгоритмами, разобранными на вебинаре.